In [1]:
import os

In [4]:
%pwd

'd:\\text_summarization'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path
     


In [6]:
from text_summariser.constants import *
from text_summariser.utils.common import read_yaml, create_directories

In [14]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
    
        self.config = read_yaml(config_filepath)    
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            tokenizer_name = config.tokenizer_name

        )

        return data_transformation_config
        

In [15]:
import os
from text_summariser.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

In [16]:
class DataTransfromation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)


    def convert_examples_to_ferature(self, example_batch):
        input_encodings = self.tokenizer(example_batch["dialogue"], truncation=True, padding="max_length", max_length=512)


        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch["summary"], truncation=True, padding="max_length", max_length=150)

        return{
            "input_ids": input_encodings['input_ids'],
            "attention_mask": input_encodings["attention_mask"],
            "labels": target_encodings["input_ids"]
        }
    

    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_ferature, batched=True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,"samusm_dataset"))

In [17]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransfromation(config = data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e


[2024-06-09 11:09:54,923] [INFO] [common.py:18] [text_summariser.logging] - Read the yaml file from config\config.yaml loaded succesfully
[2024-06-09 11:09:54,925] [INFO] [common.py:18] [text_summariser.logging] - Read the yaml file from params.yaml loaded succesfully
[2024-06-09 11:09:54,926] [INFO] [common.py:34] [text_summariser.logging] - Created directory: artifacts
[2024-06-09 11:09:54,928] [INFO] [common.py:34] [text_summariser.logging] - Created directory: artifacts/data_transformation


TypeError: __init__() got an unexpected keyword argument 'data_path'